In [2]:
#4차원 입력데이터 
import numpy as np
x = np.random.rand(10, 1, 28, 28)   #무작위로 데이터 생성
x.shape

(10, 1, 28, 28)

In [4]:
print(x[0].shape)
print(x[1].shape)

(1, 28, 28)
(1, 28, 28)


In [5]:
x[0,0]      #0.0.28.28 느낌

array([[1.63161564e-01, 5.50133707e-01, 1.23388560e-01, 1.87787631e-01,
        7.83456500e-04, 8.86955431e-01, 4.16200264e-01, 1.66333535e-01,
        8.70399364e-01, 5.34348546e-01, 5.39779861e-01, 9.13100400e-01,
        9.15824990e-01, 9.86200395e-01, 4.71019498e-01, 8.09618475e-01,
        9.81724677e-02, 8.41486967e-01, 5.02854606e-02, 7.39794659e-01,
        2.96913750e-01, 2.44136738e-01, 8.23414514e-01, 8.99873070e-01,
        2.19133915e-01, 9.00501840e-01, 6.51477298e-01, 4.12782208e-01],
       [5.56003179e-01, 8.62920385e-01, 7.14053115e-01, 2.96315263e-01,
        7.53662022e-01, 1.11080417e-01, 7.40757420e-01, 8.09510443e-01,
        2.24256784e-01, 2.08268536e-01, 3.23449459e-01, 4.85949040e-01,
        5.68145028e-02, 7.71625290e-01, 1.89170808e-01, 4.96199085e-01,
        1.50657065e-01, 2.48275567e-01, 5.38701564e-01, 2.86464735e-01,
        9.46237738e-01, 5.95479525e-01, 3.69892204e-01, 3.32128762e-01,
        8.81031268e-01, 9.67878582e-01, 7.33975288e-03, 6.31748

In [8]:
#im2col 사용해보기
import sys,os
sys.path.append(os.path.abspath('../../../'))
from common.util import im2col

x1 = np.random.rand(1,3,7,7)                        
col1 = im2col(x1, 5, 5, stride=1, pad=0)                #im2col
print(col1.shape)

x2 = np.random.rand(10, 3, 7, 7)
col2 = im2col(x2, 5, 5, stride=1, pad=0)    #5*5가 9개 있으니까
print(col2.shape)

(9, 75)
(90, 75)


In [9]:
#합성곱계층 구현하기
class Convolution:      #합성곱
    def __init__(self, W, b, stride =1 , pad =0):           #W=>가중치 == 필터안의 수 / b == 편향 
        self.W = W
        self.b = b
        self.stride = stride 
        self.pad = pad 
    
    def forward(self, x):
        #결과 크기 결정하기
        FN, C, FH, FW = self.W.shape
        N, C, H, W = x.shape
        out_h = int(1+(H + 2*self.pad - FH) / self.stride )
        out_w = int(1+ (W + 2*self.pad - FW) / self.stride)
        
        #합성곱하기
        col = im2col(x, FH, FW, self.stride, self.pad)
        col_W = self.W.reshape(FN, -1).T    #-1을 한 이유는 원소수가 변환후에도 똑같이 유지됨 im2col을 안한것처럼 보이지만 reshape로 한줄로 늘이는 과정을 한것임.
        out = np.dot(col, col_W) + self.b   
        
        #크기 되돌리기
        out = out.reshape(N, out_h, out_w, -1).transpose(0,3,1,2)   #transpose는 앞의 reshape의 순서를 정확하게 바꿔준다.
        
        return out

In [11]:
#풀링계층 구현하기
class Pooling:
    def __init__(self, pool_h, pool_w, stride=1, pad=0):
        self.pool_h = pool_h
        self.pool_w = pool_w
        self.stride = stride
        self.pad = pad
        
    def forward(self, x):
        N, C, H, W = x.shape
        out_h = int(1+ (H - self.pool_h) / self.stride)
        out_w = int(1+ (W - self.pool_w) / self.stride)
        
        #전개
        col = im2col(x, self.pool_h, self.pool_w, self.stride, self.pad)
        col = col.reshape(-1, self.pool_h*self.pool_w)
        
        #최댓값
        out = np.max(col,axis=1)
        
        #성형
        out = out.reshape(N, out_h, out_w, C).transpose(0, 3, 1,2)      
        
        return out

In [17]:
class SimpleConNet:
    def __init__(self, input_dim=(1, 28, 28),
                conv_param = {'filter_num':30, 'filter_size':5,
                            'pad':0, 'stride':1},
                hidden_size =100, output_size = 10, weight_init_std = 0.01):
        filter_num = conv_param['filter_num']
        filter_size = conv_param['filter_size']
        filter_pad = conv_param['pad']
        filter_stride = conv_param['stride']
        input_size = input_dim[1]
        conv_output_size = (input_size - filter_size + 2*filter_pad) / \
                            filter_stride + 1
        pool_output_size = int(filter_num * (conv_output_size/2) * 
                               (conv_output_size / 2))    
        
        #
        self.params = {}
        self.parmas['W1'] = weight_init_std * \
            np.random.randn(filter_num, input_dim[0],
                            filter_size, filter_size)
        self.params['b1'] = np.zeros(filter_num)
        self.params['W2'] = weight_init_std * \
            np.random.randn(pool_output_size,
                            hidden_size)
        self.params['b2'] = np.zeros(hidden.size)
        self.parmas['W3'] = weight_init_std *\
            np.random.randn(hidden_size, output_size)
        self.params['b3'] = np.zeros(output_size)
        
        #
        self.layers = OrderedDict()
        self.layers['Conv1'] = Convolution(self.params['W1'],
                                           self.parmas['b1'],
                                           conv_params['stride'],
                                           conv_params['pad'])
        self.layers['Relu1'] = Relu()
        self.layers['Pool1'] = Pooling(pool_h=2, pool_w=2,stride=2)
        self.layers['Affine1'] = Affine(self.params['W2'],
                                        self.params['b2'])
        self.layers['Relu2'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W3'],
                                        self.params['b3'])
        self.last_layer = SoftmaxWithLoss()
        
    # 
    def predict(self, x):
        for layer in self.layers.values():
            return x
    def loss(self, x,t):
        y = self.predict(x)
        return self.last_layer.forward(y,t)
    
    #
    def gradient(self,x,t):
        #순전파
        self.loss(x,t)
        
        #역전파
        dout = 1
        dout = self.last_layer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        for layer in layers:
            dout = layer.backward(dout)
        
        #결과저장
        grads = {}
        grads['W1'] = self.layers['Conv1'].dW
        grads['b1'] = self.layers['Conv1'].db
        grads['W2'] = self.layers['Affine1'].dW
        grads['b2'] = self.layers['Affine1'].db
        grads['W3'] = self.layers['Affine2'].dW
        grads['b3'] = self.layers['Affine2'].db
        
        return grads